In [2]:
import pandas as pd
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
import requests
import datetime
import json

# Wczytanie danych GTFS-RT
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://gtfs.ztp.krakow.pl/VehiclePositions_A.pb')
feed.ParseFromString(response.content)


entity_dict = MessageToDict(feed.entity[0], preserving_proto_field_name=True)

# Ładne wyświetlenie JSON
print(json.dumps(entity_dict, indent=2))
# Lista na dane
vehicles = []

# Przetwarzanie danych
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle = entity.vehicle

        vehicles.append({
            'vehicle_id': vehicle.vehicle.id,
            'trip_id': vehicle.trip.trip_id,
            'route_id': vehicle.trip.route_id,
            'latitude': vehicle.position.latitude,
            'longitude': vehicle.position.longitude,
            'bearing': vehicle.position.bearing,
            'speed': vehicle.position.speed,
            'timestamp': datetime.datetime.fromtimestamp(vehicle.timestamp),
            'stop_id':vehicle.stop_id,
            'stop_sequence':vehicle.current_stop_sequence,
            'current_date': datetime.date.today(),
            'day': datetime.date.today().isoweekday()
        })

# Konwersja do DataFrame
df = pd.DataFrame(vehicles)

# Zapis do CSV (opcjonalnie)
df.to_csv('vehicle_positions.csv', index=False)

# Podgląd
print(df.head())

{
  "id": "2",
  "vehicle": {
    "trip": {
      "trip_id": "block_194_trip_8_service_1"
    },
    "position": {
      "latitude": 50.021397,
      "longitude": 19.960312
    },
    "current_stop_sequence": 8,
    "current_status": "STOPPED_AT",
    "timestamp": "1746617611",
    "stop_id": "stop_446_62903",
    "vehicle": {
      "id": "2",
      "label": "2",
      "license_plate": "DN002"
    }
  }
}
  vehicle_id                     trip_id route_id   latitude  longitude  \
0          2  block_194_trip_8_service_1           50.021397  19.960312   
1          7  block_170_trip_7_service_1           50.048958  19.931528   
2          8   block_98_trip_7_service_1           50.092510  19.921980   
3          9  block_104_trip_6_service_1           50.006340  20.010555   
4         11  block_100_trip_7_service_1           50.079983  19.918507   

   bearing  speed           timestamp           stop_id  \
0      0.0    0.0 2025-05-07 13:33:31    stop_446_62903   
1    180.0    0.0 2025

In [45]:
import pandas as pd
import opoznienie
from google.transit import gtfs_realtime_pb2
import requests
import datetime
import os

ref_df=pd.read_csv('../data/rozklad/stop_times.txt',delimiter=',')
positions_df=pd.read_csv('../data/positions/vehicle_positions_2025-04-30_004744.csv')

merged = pd.merge(positions_df,ref_df[['trip_id', 'stop_id', 'departure_time','stop_sequence']],on=['trip_id', 'stop_id','stop_sequence'],how='left')

merged['timestamp'] = pd.to_datetime(merged['timestamp'])


merged['planned_departure'] = merged.apply(
    lambda row: opoznienie.parse_departure_time_final(row['departure_time'], row['timestamp']) if pd.notnull(row['departure_time']) else pd.NaT,
    axis=1
)


# Obliczanie opóźnienia (w minutach)
merged['delay_sec'] = (merged['timestamp'] - merged['planned_departure']).dt.total_seconds()

merged.head()

-86311.0
-86381.0
-88774.0


,vehicle_id,trip_id,route_id,latitude,longitude,speed,timestamp,stop_id,stop_sequence,current_status,current_date,day,departure_time,planned_departure,delay_sec
0,320,block_542_trip_11_service_1,NaN,50.174305,19.913160,0.0,2025-04-30 00:47:29,stop_1313_296801,1,1,2025-04-30,3,24:46:00,2025-04-30 00:46:00,89.0
1,508,block_509_trip_10_service_1,NaN,50.062763,19.923124,0.0,2025-04-30 00:47:19,stop_1626_311102,18,1,2025-04-30,3,24:47:00,2025-04-30 00:47:00,19.0
2,594,block_524_trip_3_service_1,NaN,50.100731,20.013161,0.0,2025-04-30 00:47:26,stop_273_37802,1,1,2025-04-30,3,25:27:00,2025-04-30 01:27:00,-2374.0


In [12]:
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
import requests
import json

# Wczytanie danych GTFS-RT
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://gtfs.ztp.krakow.pl/TripUpdates_A.pb')
feed.ParseFromString(response.content)

entity_dict = MessageToDict(feed.entity[0], preserving_proto_field_name=True)

# Ładne wyświetlenie JSON
print(json.dumps(entity_dict, indent=2))

# Lista na dane
vehicles = []

# Przetwarzanie danych
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle = entity.vehicle

        vehicles.append({
            'trip_id': vehicle.trip_update.id,
            ''
        })

# Konwersja do DataFrame
df = pd.DataFrame(vehicles)

# Zapis do CSV (opcjonalnie)
df.to_csv('vehicle_positions.csv', index=False)

# Podgląd
print(df.head())

{
  "id": "block_974_trip_17",
  "trip_update": {
    "trip": {
      "trip_id": "block_974_trip_17_service_3"
    },
    "stop_time_update": [
      {
        "departure": {
          "time": "1745777400"
        },
        "stop_id": "stop_36_5401"
      },
      {
        "arrival": {
          "time": "1745777520"
        },
        "departure": {
          "time": "1745777520"
        },
        "stop_id": "stop_580_82202"
      },
      {
        "arrival": {
          "time": "1745777640"
        },
        "departure": {
          "time": "1745777640"
        },
        "stop_id": "stop_7_1203"
      },
      {
        "arrival": {
          "time": "1745777700"
        },
        "departure": {
          "time": "1745777700"
        },
        "stop_id": "stop_2309_345907"
      },
      {
        "arrival": {
          "time": "1745777760"
        },
        "departure": {
          "time": "1745777760"
        },
        "stop_id": "stop_105_13701"
      },
      {
        "

In [1]:
import pandas as pd
import opoznienie
from google.transit import gtfs_realtime_pb2
import requests
import datetime
import os

test=pd.DataFrame()
print(test.empty)


delays=pd.read_csv('../data/opoznienia/delays.csv')


#print(delays.axes)

delays_new=delays[['vehicle_id','trip_id','stop_sequence','current_status','day', 'planned_departure','timestamp','delay_sec']]
delays_new['delay_min']=delays_new['delay_sec']/60

delays_new.head()


True


C:\Users\janus\AppData\Local\Temp\ipykernel_9656\1277319170.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delays_new['delay_min']=delays_new['delay_sec']/60


,vehicle_id,trip_id,stop_sequence,current_status,day,planned_departure,timestamp,delay_sec,delay_min
0,2,block_194_trip_9_service_1,17,1,3,2025-05-07 14:44:00,2025-05-07 14:51:11,431.0,7.183333
1,8,block_98_trip_8_service_1,1,1,3,2025-05-07 14:57:00,2025-05-07 14:51:41,-319.0,-5.316667
2,19,block_168_trip_11_service_1,1,1,3,2025-05-07 14:58:00,2025-05-07 14:51:42,-378.0,-6.300000
3,22,block_175_trip_11_service_1,15,1,3,2025-05-07 14:52:00,2025-05-07 14:51:38,-22.0,-0.366667
4,27,block_197_trip_9_service_1,7,1,3,2025-05-07 14:50:00,2025-05-07 14:51:43,103.0,1.716667


In [7]:
delays_new.loc[delays_new['vehicle_id']==2]


,vehicle_id,trip_id,stop_sequence,current_status,day,planned_departure,timestamp,delay_sec,delay_min
0,2,block_194_trip_9_service_1,17,1,3,2025-05-07 14:44:00,2025-05-07 14:51:11,431.0,7.183333
82,2,block_194_trip_9_service_1,17,1,3,2025-05-07 14:44:00,2025-05-07 14:51:11,431.0,7.183333
